In [ ]:
!pip install diffusers omegaconf transformers accelerate safetensors invisible_watermark xformers compel
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:32

from IPython.display import clear_output
clear_output()

In [ ]:
!curl -L "https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/diffusion_pytorch_model.safetensors" --output diffusion_pytorch_model.safetensors
!curl -L "https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/config.json" --output config.json
!mkdir monsterv2
!mv diffusion_pytorch_model.safetensors config.json monsterv2

!curl -L "https://civitai.com/api/download/models/9208?type=Model&format=SafeTensor&size=full&fp=fp16" --output easynegative.safetensors
!curl -L "https://civitai.com/api/download/models/5637?type=Model&format=PickleTensor&size=full&fp=fp16" --output ng_deepnegative_v1_75t.pt

In [ ]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline, DPMSolverMultistepScheduler
from omegaconf import OmegaConf
from diffusers.utils import load_image


controlnet = ControlNetModel.from_pretrained("Nacholmo/controlnet-qr-pattern-v2")
# controlnet = ControlNetModel.from_pretrained("monster-labs/control_v1p_sd15_qrcode_monster")
# controlnet = ControlNetModel.from_pretrained("/content/monsterv2")

# Realistic
# pipeline = StableDiffusionControlNetPipeline.from_single_file(
# 	"https://huggingface.co/Lykon/DreamShaper/blob/main/DreamShaper_8_pruned.safetensors",
# 	controlnet=controlnet,
# 	use_safetensors=True
# )

# Anime Style
pipeline = StableDiffusionControlNetPipeline.from_single_file(
	"https://huggingface.co/Meina/MeinaMix/blob/main/Meina V10 - baked VAE.safetensors",
	controlnet=controlnet,
	use_safetensors=True
)

pipeline.to('cuda')

from IPython.display import clear_output
clear_output()

In [ ]:
controlnet.save_pretrained("/content/test")

In [ ]:
import hashlib

with open("/content/test/diffusion_pytorch_model.safetensors","rb") as f:
    bytes = f.read() # read entire file as bytes
    readable_hash = hashlib.sha256(bytes).hexdigest();
    print(readable_hash)

c7debd28037965262e1696d3a96fc9309323872da00db84b787b45058a3b1dc7


In [ ]:
pipeline.load_textual_inversion("/content/easynegative.safetensors")
pipeline.load_textual_inversion("/content/ng_deepnegative_v1_75t.pt")
pipeline.load_lora_weights("OedoSoldier/detail-tweaker-lora")

In [ ]:
from compel import Compel, DiffusersTextualInversionManager
import torch

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, use_karras_sigmas=True)



qr_code_image = load_image(
    "/content/qr_code.png"
)

prompt = f"""
(Masterpiece)++, (Illustration)++, (Exquisite Beauty)++
(masterpiece), Beautiful sky from top of the clouds,
fluffy clouds, clouds flowting near the mountain, nijo,
Inspire from your name anime, High green mountains,
High detailed, sherp image"""


nprompt = """

"""

fix_nprompt = """,<easynegative>+, <ng_deepnegative_v1_75t>+,(worst quality, low quality)++, monochrome, zombie,overexposure, watermark,text,bad anatomy,
bad hand,extra hands,extra fingers,too many fingers,fused fingers,bad arm,distorted arm,
extra arms,fused arms,extra legs,missing leg,disembodied leg,extra nipples, detached arm,
liquid hand,inverted hand,disembodied limb, small breasts, loli, oversized head,extra body,
completely nude, extra navel,easynegative,(hair between eyes)+,sketch, duplicate, ugly,
huge eyes, text, logo, worst face, (bad and mutated hands)++, (blurry)++, horror, geometry,
bad_prompt, (bad hands)+, (missing fingers)+, multiple limbs, bad anatomy, (interlocked fingers:)++,
Ugly Fingers, (extra digit and hands and fingers and legs and arms:)+,
(deformed fingers)+, (long fingers)+,(bad-artist-anime)+, bad-artist, bad hand,
extra legs ,(ng_deepnegative_v1_75t), tree+++"""

textual_inversion_manager = DiffusersTextualInversionManager(pipeline)

compel = Compel(
    tokenizer=pipeline.tokenizer, text_encoder=pipeline.text_encoder,
    textual_inversion_manager=textual_inversion_manager)

conditioning = compel.build_conditioning_tensor(prompt)
nconditioning = compel.build_conditioning_tensor(nprompt+fix_nprompt)

image = pipeline(prompt_embeds=conditioning, negative_prompt_embeds =nconditioning,
                 image=qr_code_image,
                 num_inference_steps=30,
                 controlnet_conditioning_scale=0.8,
                 control_guidance_start=0,control_guidance_end=0.8,
                 cross_attention_kwargs={"scale": 1},
                 width=1024,height=1024).images[0]


torch.cuda.empty_cache()
from IPython.display import Image, display
display(image)